<a href="https://colab.research.google.com/github/martine-augustin/Marionnaud/blob/main/Marionnaud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Marionnaud projet


## Variable et installation

In [15]:
# Install
!pip install beautifulsoup4
!pip install lxml




In [16]:
# Import
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np


## Scrapping

In [18]:
# 1. L'URL de la page à scraper
url = 'https://fr.trustpilot.com/review/www.marionnaud.com'

# 2. Récupérer le contenu de la page avec 'requests'
response = requests.get(url)

In [19]:
# Vérifier si la requête a réussi (Code 200)
soup = None
if response.status_code == 200:

    # 3. Créer l'objet BeautifulSoup pour parser le HTML
    # 'html.parser' est le parser inclus dans Python
    soup = BeautifulSoup(response.text, 'html.parser')

else:
    print("Erreur lors de la récupération de la page.")

In [22]:
 # --- EXEMPLE 1 : Récupérer le titre de la page ---
titre = soup.title.string
print(f"Titre de la page : {titre}\n")

# --- EXEMPLE 2 : Trouver un élément spécifique (le premier h1) ---
h1 = soup.find('h1')
print(f"Titre H1 : {h1.text}")

# --- EXEMPLE 3 : Trouver TOUS les liens (balises <a>) ---
print("\n--- Liste des liens ---")
liens = soup.find_all('p') # Trouve toutes les balises <a>

for lien in liens:
  # On récupère le texte du lien et son URL (href)
  texte_du_lien = lien.text
  url_du_lien = lien.get('href')
  print(f"Texte : {texte_du_lien} -> Lien : {url_du_lien}")

Titre de la page : Avis de Marionnaud France | Lisez les avis marchands de www.marionnaud.com

Titre H1 : Marionnaud France Avis 7 307

--- Liste des liens ---
Texte : Profil revendiqué -> Lien : None
Texte : 3,7 -> Lien : None
Texte : Bien que nous ne vérifiions pas les affirmations individuelles, car les avis reflètent l’opinion personnelle des utilisateurs, certains avis sont accompagnés du statut « vérifié » lorsqu’il est possible de confirmer qu’une interaction avec l’entreprise a eu lieu. En savoir plus -> Lien : None
Texte : Pour protéger l’intégrité de la plateforme, chaque avis, qu’il soit vérifié ou non, est analysé par notre logiciel automatisé 24 h/24 et 7 j/7. Il identifie et retire tout contenu qui enfreint nos conditions d'utilisation, y compris les avis qui ne se basent pas sur des expériences authentiques. Nous sommes conscients que nous ne pouvons pas tout détecter, et vous pouvez nous signaler tout élément qui aurait pu nous échapper. En savoir plus -> Lien : None
Te